In [ ]:
import geocoder
import ephem
import ephem.stars
import datetime
import math
import cmath
import sympy
import functools
import numpy as np
import qutip
import random
import vpython
import mpmath
import scipy

def latlng_xyz(latitude, longitude):
    x = math.sin(latitude)*math.cos(longitude)
    y = math.sin(latitude)*math.sin(longitude)
    z = math.cos(latitude)
    return [x, y, z]

def altaz_xyz(altitude, azimuth):
    x = math.sin(azimuth)*math.cos(altitude)
    y = math.cos(altitude)*math.cos(azimuth)
    z = math.sin(altitude)
    return [x, y, z]

def xyz_c(x, y, z):
    if z == 1:
        return float('inf') 
    else:
        return complex(x/(1-z), y/(1-z))

def c_xyz(c):
    if c == float('inf'):
        return [0,0,1]
    x = c.real
    y = c.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def roots_polynomial(roots):
    s = sympy.symbols("s")
    polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s+root for root in roots]), domain="CC")
    return polynomial.coeffs()

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

def polynomial_state(polynomial):
    coordinates = [polynomial[i]/(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) for i in range(len(polynomial))]
    return qutip.Qobj(np.array(coordinates).T)

def distinguish_qubits(state):
    state.dims = [[2,2,2],[1,1,1]]
    traces = [state.ptrace(i).full() for i in range(3)]
    state.dims = [[8],[1]]
    return traces

def scalar_product(m, n):
    return 0.5*np.trace(np.dot(np.conjugate(m).T, n))

def qubit_txyz(m):
    t = scalar_product(np.eye(2), m).real
    x = scalar_product(qutip.sigmax().full(), m).real
    y = scalar_product(qutip.sigmay().full(), m).real
    z = scalar_product(qutip.sigmaz().full(), m).real
    return [t/math.sqrt(t**2+x**2+y**2+z**2), x/t, y/t, z/t]

def state_polynomial(v):
    polynomial = v.T.tolist()
    return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))]

def polynomial_roots(polynomial):
    try:
        roots = [complex(root) for root in mpmath.polyroots(polynomial)]
    except:
        roots = []
    return roots

STAR_NAMES = [star.split(",")[0] for star in ephem.stars.db.split("\n")][:-1]

class Sphere:
    def __init__(self, address, time, time_delta):
        self.flag = True
        self.latitude, self.longitude = geocoder.google(address).latlng
        self.time = time
        self.time_delta = time_delta
        
        self.state = None
        self.new_state = None
        #self.three_operators = [qutip.rand_unitary(2), qutip.rand_unitary(2), qutip.rand_unitary(2)]
        #self.three_operators = [qutip.rand_herm(2), qutip.rand_herm(2), qutip.rand_herm(2)]
        self.three_operators = [qutip.sigmax(), qutip.sigmay(), qutip.sigmaz()]
        
        vpython.scene.width = 1000
        vpython.scene.height = 1000
        vpython.scene.range = 1.5
        vpython.scene.forward = vpython.vector(0, 1, 0)
        vpython.scene.up = vpython.vector(-1, 0, 0)
        
        self.vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)
        self.vearth = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.1, color=vpython.color.cyan, opacity=0.5, emissive=True)
        self.vobserver = vpython.sphere(pos=vpython.vector(0,0,0), radius=0.01, color=vpython.color.yellow, opacity=0.5, emissive=True, make_trail=True)
        
        self.vstamp = vpython.label(pos=vpython.vector(0,0,0), text="", height=10, opacity=0.6)

        self.vfixed_stars = [vpython.sphere(color=vpython.color.white, radius=0.01, emissive=True, make_trail=False) for i in range(len(STAR_NAMES))]
        self.vfixed_stars[STAR_NAMES.index("Sirius")].color = vpython.color.red
        self.vfixed_stars[STAR_NAMES.index("Betelgeuse")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Rigel")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Bellatrix")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Mintaka")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Alnilam")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Alnitak")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Saiph")].color = vpython.color.yellow
        self.vfixed_stars[STAR_NAMES.index("Polaris")].color = vpython.color.blue

        self.vplanets = [vpython.sphere(radius=0.1, emissive=True, make_trail=False) for i in range(7)]
        self.vplanets[0].color = vpython.color.yellow
        self.vplanets[1].color = vpython.color.white
        self.vplanets[2].color = vpython.color.blue
        self.vplanets[3].color = vpython.color.green
        self.vplanets[4].color = vpython.color.red
        self.vplanets[5].color = vpython.color.orange
        self.vplanets[6].color = vpython.color.gray(0.5)
        
        self.vqubits = [vpython.sphere(radius=0.1, emissive=True, make_trail=False) for i in range(3)]
        self.veigs = [vpython.sphere(color=vpython.color.black, radius=0.05, emissive=True, make_trail=False) for i in range(6)]
        self.vlines = [vpython.curve(pos=[vpython.vector(0,0,0), vpython.vector(0,0,0)]) for i in range(3)]

    def draw(self, fixed_stars_XYZ, planets_XYZ, qubits_TXYZ, eigs_XYZ):
        vpython.rate(300)        
        self.vobserver.pos = vpython.vector(*[0.1*coord for coord in latlng_xyz(self.latitude, self.longitude)])
        self.vstamp.text = self.time.strftime("%c")+"\nlat: %.2f lng: %.2f" % (self.latitude, self.longitude)
        for i in range(len(self.vfixed_stars)):
            self.vfixed_stars[i].pos = vpython.vector(*fixed_stars_XYZ[i])
        for i in range(len(self.vplanets)):
            self.vplanets[i].pos = vpython.vector(*planets_XYZ[i])
        for i in range(len(self.vqubits)):
            self.vqubits[i].color = vpython.color.hsv_to_rgb(vpython.vector(float(qubits_TXYZ[i][0]),1,1))
            self.vqubits[i].pos = vpython.vector(*qubits_TXYZ[i][1:])
        for i in range(len(self.veigs)):
            self.veigs[i].pos = vpython.vector(*eigs_XYZ[i])
        for i in range(len(self.vlines)):
            self.vlines[i].modify(0, pos=vpython.vector(*eigs_XYZ[2*i]))
            self.vlines[i].modify(1, pos=vpython.vector(*eigs_XYZ[2*i+1]))

    def observer(self):
        observer = ephem.Observer()
        observer.date = self.time
        observer.lat = self.latitude
        observer.lon = self.longitude
        return observer

    def planets(self, observer):
        return [ephem.Sun(observer), ephem.Moon(observer), ephem.Mercury(observer),\
                ephem.Venus(observer), ephem.Mars(observer), ephem.Jupiter(observer),\
                ephem.Saturn(observer)]

    def fixed_stars(self, observer):
        return [ephem.star(star_name, observer) for star_name in STAR_NAMES]

    def revolve(self):
        if(self.flag):
            observer = self.observer()
            fixed_stars = self.fixed_stars(observer)
            fixed_stars_XYZ = [altaz_xyz(fixed_star.alt, fixed_star.az) for fixed_star in fixed_stars]

            planets = self.planets(observer)
            planets_XYZ = [altaz_xyz(planet.alt, planet.az) for planet in planets]
            planets_C = [xyz_c(*xyz) for xyz in planets_XYZ]

            self.state = polynomial_state(roots_polynomial(planets_C))
            distinguishable_qubits = distinguish_qubits(self.state)
            qubits_TXYZ = [qubit_txyz(qubit) for qubit in distinguishable_qubits]

            eigs_XYZ = []
            for qubit in distinguishable_qubits:
                eigenvalues, eigenvectors = np.linalg.eig(qubit)
                for v in eigenvectors:
                    eigs_XYZ.append(c_xyz(polynomial_roots(state_polynomial(np.conjugate(v)))[0]))

            self.draw(fixed_stars_XYZ, planets_XYZ, qubits_TXYZ, eigs_XYZ)
            self.time += self.time_delta
        else:
            if self.new_state != None:
                planets_XYZ = [c_xyz(root) for root in polynomial_roots(state_polynomial(self.new_state.full().T[0]))]
                distinguishable_qubits = distinguish_qubits(self.new_state)
                qubits_TXYZ = [qubit_txyz(qubit) for qubit in distinguishable_qubits]
                eigs_XYZ = []
                for q in distinguishable_qubits:
                    eigenvalues, eigenvectors = np.linalg.eig(q)
                    for v in eigenvectors:
                        eigs_XYZ.append(c_xyz(polynomial_roots(state_polynomial(np.conjugate(v)))[0]))
                vpython.rate(300)  
                for i in range(len(self.vplanets)):
                    self.vplanets[i].pos = vpython.vector(*planets_XYZ[i])
                for i in range(len(self.vqubits)):
                    self.vqubits[i].color = vpython.color.hsv_to_rgb(vpython.vector(float(qubits_TXYZ[i][0]),1,1))
                    self.vqubits[i].pos = vpython.vector(*qubits_TXYZ[i][1:])
                for i in range(len(self.veigs)):
                    self.veigs[i].pos = vpython.vector(*eigs_XYZ[i])
                for i in range(len(self.vlines)):
                    self.vlines[i].modify(0, pos=vpython.vector(*eigs_XYZ[2*i]))
                    self.vlines[i].modify(1, pos=vpython.vector(*eigs_XYZ[2*i+1]))
    
    def reset_vqubits(self):
        for vqubit in self.vqubits:
            vqubit.clear_trail()
        
    def latitude_step(self, n):
        step = 0.01
        self.latitude += step*n
        if self.latitude >= 90:
            self.latitude -= 90
        if self.latitude <= -90:
            self.latitude += 90
        self.reset_vqubits()
    
    def longitude_step(self, n):
        step = 0.01
        self.longitude += step*n
        if self.longitude >= 180:
            self.longitude -= 180
        if self.longitude <= -180:
            self.longitude += 180
        self.reset_vqubits()
    
    def rate_step(self, n):
        step = 60
        self.time_delta = datetime.timedelta(seconds=(self.time_delta.seconds+n*step))
    
    def qubit_step(self, i_qubit, n):
        self.flag = False
        distinguishable_qubits = None
        if self.new_state != None:
            distinguishable_qubits = distinguish_qubits(self.new_state)
        else:
            distinguishable_qubits = distinguish_qubits(self.state)
        #qubit = qutip.Qobj(distinguishable_qubits[i_qubit])
        #eigenvalues, eigenvectors = qubit.eigenstates()
        #destination = None
        #if n == -1:
        #    destination = eigenvectors[0]
        #elif n == 1:
        #    destination = eigenvectors[1]
        #operator = qutip.tensor(destination, destination.dag())*qubit.dag()
        #operator = qutip.Qobj((np.conjugate(qubit).T/np.linalg.norm(qubit)**2)*destination)
        #full_operator = None
        #if i_qubit == 0:
        #    full_operator = np.outer(operator, np.eye(2))
        #    full_operator = np.outer(full_operator, np.eye(2))
        #elif i_qubit == 1:
        #    full_operator = np.outer(np.eye(2), operator)
        #    full_operator = np.outer(full_operator, np.eye(2))
        #elif i_qubit == 2:
        #    full_operator = np.outer(np.eye(2), np.eye(2))
        #    full_operator = np.outer(full_operator, operator)
        #full_unitary = scipy.linalg.expm(-2*math.pi*complex(0,1)*full_operator)
        #self.state = Qobj(full_unitary.dot(self.state))
        #operator = qutip.rand_herm(2)
        #operator = self.three_operators[i_qubit]
        #operator = distinguishable_qubits[i_qubit]
    
        #operator = distinguishable_qubits[0]
        operator = None
        if i_qubit == 0:
            operator = distinguishable_qubits[1]
        elif i_qubit == 1:
            operator = distinguishable_qubits[2]
        elif i_qubit == 2:
            operator = distinguishable_qubits[0]
        operator = qutip.Qobj(scipy.linalg.expm(-2*math.pi*complex(0,1)*operator*0.0005))
        if n == -1:
            operator = operator.dag()
        full_operator = None
        if i_qubit == 0:
            full_operator = qutip.tensor(operator, qutip.identity(2), qutip.identity(2))
        elif i_qubit == 1:
            full_operator = qutip.tensor(qutip.identity(2), operator, qutip.identity(2))
        elif i_qubit == 2:
            full_operator = qutip.tensor(qutip.identity(2), qutip.identity(2), operator)
        full_operator.dims = [[8],[8]]
        if self.new_state == None:
            self.new_state = full_operator*self.state
        else:
            self.new_state = full_operator*self.new_state
        self.reset_vqubits()
        
sphere = Sphere("476 Jefferson Street, Brooklyn", datetime.datetime.now(), datetime.timedelta(minutes=0))

def keyboard(event):
    global sphere
    key = event.key
    if key == "a": 
        sphere.latitude_step(-1)
    elif key == "d": 
        sphere.latitude_step(1)
    elif key == "w": 
        sphere.longitude_step(1)
    elif key == "s": 
        sphere.longitude_step(-1)
    elif key == "z": 
        sphere.rate_step(-1)
    elif key == "x": 
        sphere.rate_step(1)
    elif key == "j":
        sphere.qubit_step(0, -1)
    elif key == "l":
        sphere.qubit_step(0, 1)
    elif key == "i":
        sphere.qubit_step(1, 1)
    elif key == "k":
        sphere.qubit_step(1, -1)
    elif key == "n":
        sphere.qubit_step(2, -1)
    elif key == "m":
        sphere.qubit_step(2, 1)
        
vpython.scene.bind('keydown', keyboard)

while True:
    sphere.revolve()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>